In [1]:
# -- setup --

# Connection
import snowmobile
sn = snowmobile.connect()

# Find all code files
from pathlib import Path

snippets = Path.cwd().parent / 'snippets'
scripts = {p.name: p for p in snippets.glob('**/*.sql')}

Locating credentials..
(1 of 2) Finding snowmobile.toml..
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Snowmobile(creds='creds1')


(sql)=
# SQL
<hr class="sn-grey">
<a 
    class="sphinx-bs badge badge-primary text-white reference external sn-api sn-link-container2" 
    href="../autoapi/snowmobile/core/sql/index.html" 
    title="API Documentation">
    <span>snowmobile.core.sql</span>
</a>

The {class}`~snowmobile.core.connection.Snowmobile.sql` attribute of
<a class="fixture-sn" href="../index.html#fixture-sn"></a>
is a {class}`~snowmobile.core.sql.SQL` object that generates and 
executes raw SQL from inputs; its purpose is to provide a simple, on-hand Python
API for querying metadata and executing basic administrative commands against {xref}`snowflake`.

By default, {class}`~snowmobile.core.connection.Snowmobile.sql` will
execute the generated sql and return its results; execution can be omitted
and the generated sql returned as a raw string by providing *run=False* 
to the method being invoked or by manually setting its 
{attr}`~snowmobile.SQL.auto_run` attribute to *False* prior to calling the 
method.

<hr class="sn-grey">


````{admonition} Warning
 :class: warning, toggle, toggle-shown
 
 ```{div} sn-pre-code-s
 (sql/warning)=
 These methods will not ask twice before querying, altering or dropping a 
 {xref}`snowflake` object; isolated testing to ensure the API
 is understood before use is recommended.
 
 Providing *run=False* and printing the returned string to the console is one 
 of the easiest ways to inspect the sql that's generated by a given method.
 ```
 
 ```python
 print(sn.sql.drop('sample_table', run=False))
 """
 >>>
 drop table if exists sample_table
 """
 ```
 
 <hr class="sn-spacer-thin">

````

<hr class="sn-spacer-thick">

(setup)=
## Usage

```{div} sn-dedent-list, sn-depth1
- [](#command-overview)
- [](#execution-control)
- [](usage/sql/setting-nm-and-obj)
```

<br>
<br>

 ````{admonition} Setup
 :class: is-setup, sn-clear-title

 These examples make use\
 of a **sample_table** containing:
 ```{div} sn-dedent-v-b-container
 |   COL1 |   COL2 |
 |-------:|-------:|
 |      1 |      1 |
 |      2 |      4 |
 |      4 |      9 |
 ```
 ````

<hr class="sn-spacer-thick">

### Command Overview
<hr class="sn-green-thick-g">

````````{div} sn-tabbed-section

 ```````{tabbed} -
 
 ```{admonition} FYI
 :class: tip, sn-clear-title
 The snippets below encompass the most widely applicable methods available off 
 [snowmobile.SQL](#sql); see
 <a href="../autoapi/snowmobile/core/sql/index.html" style="display: inline">
        <span>the API Docs</span>
 </a>
 for exhaustive method documentation.
 ```
 
 *The following statements can be run to interact with* **sample_table** *defined by* [{fa}`cog`](setup).
 
 <hr class="sn-spacer2">
 <hr class="sn-green-thin">
 
 ```{div} sn-pre-code-s, sn-post-code
 Verify it exists:
 ```
 ```python
 sn.sql.exists('sample_table')  #> True
 ```
 
 ```{div} sn-pre-code-s, sn-post-code
 Sample its records:
 ```
 ```python
 sn.sql.select('sample_table', n=1)
 ```
 
 ```{div} sn-pre-code-s, sn-post-code
 Query its columns from selecting a sample record:
 ```
 ```python
 sn.sql.columns('sample_table')  #> ['COL1', 'COL2']
 ```
 ```{div} sn-pre-code-s
 Or from the information schema:
 ```
 ```python
 sn.sql.columns('sample_table', from_info_schema=True)  #> ['COL1', 'COL2']
 ```
 
 ```{div} sn-pre-code-s, sn-post-code
 Check its depth:
 ```
 ```python
 sn.sql.count('sample_table')  #> 3
 ```

 ```{div} sn-pre-code-s, sn-post-code
 Query its DDL:
 ```
 ```python
 print(sn.sql.ddl('sample_table'))
 """
 >>>
 create or replace TABLE SAMPLE_TABLE (
   COL1 FLOAT,
   COL2 FLOAT
 );
 """
 ```

 ```{div} sn-pre-code-s, sn-post-code
 Clone it to another table:
 ```
 ```python
 sn.sql.clone(nm='sample_table', to='sample_table2')
 ```

 ```{div} sn-pre-code-s
 Drop objects:
 ```
 ```python
 sn.sql.drop('sample_table2')
 sn.sql.exists('sample_table2')  #> False
 ```
 ```````
 
 ```````{tabbed} Cross-Schema
 Applicable methods of {attr}`~snowmobile.core.connection.Snowmobile.sql` inspect the value 
 passed through the `nm` argument for schema-prefixes; when provided, 
 `sn.sql` will compare the schema passed as an argument to the schema associated with
 <a class="fixture-sn" href="../index.html#fixture-sn"></a> in order to generate the 
 appropriate sql.
 
 <hr class="sn-spacer2">
 <hr class="sn-green-thin">

 For example, if `other_schema` represents a different schema than
 <a class="fixture-sn" href="../index.html#fixture-sn"></a> is currently connected
 to, the following two statements could be run:
 
 ```{div} sn-pre-code-s
 Clone `sample_table` to `other_schema.sample_table`:
 ```
 ```python
 sn.sql.clone(nm='sample_table', to='other_schema.sample_table')
 ```
 
 ```{div} sn-pre-code-s, sn-post-code
 Drop `other_schema.sample_table` from the current schema:
 ```
 ```python
 sn.sql.drop(nm='other_schema.sample_table')
 ```

 ```````

````````

<hr class="sn-spacer-thick">

### Execution Control
<hr class="sn-green-thick-g">

````````{div} sn-tabbed-section

 ```````{tabbed} &nbsp;
 
 <h4><em>run=False</em></h4>
 <hr class="sn-green-medium">
 
 ```{div} sn-pre-code-s
 (using-auto-run)=
 Also demonstrated [above](sql/warning), methods can be provided with *run=False* to return 
 the raw sql as a string as opposed to executing the generated command:
 ```
 ```python
 print(sn.sql.drop('sample_table', run=False))
 """
 >>>
 drop table if exists sample_table
 """

 print(sn.sql.select('sample_table', n=1))
 """
 >>>
 select
     *
 from sample_table
 limit 1
 """
 ```
 
 <br>
 <h4>Using <code>auto_run</code></h4>
 <hr class="sn-green-medium">
 
 The `run` method argument has the following signature:
 ```{div} sn-left-pad2
 *run: Optional[*{class}`bool`*] = None*
 ```
 If a valid {class}`bool` isn't passed in the place of *None*, the current value of its
 {attr}`~snowmobile.core.sql.SQL.auto_run` attribute determines whether or not
 to execute the sql it generates.
 
 ```{div} sn-pre-code-s
 An alternative to providing *run=False* across a series of methods in order to inspect the sql being
 generated is then to modify this attribute's value on a given instance of 
 <a class="fixture-sn" href="../index.html#fixture-sn"></a>, done with:
 ```
 ```python
 sn.sql.auto_run = False
 ```
 
 ```{div} sn-pre-code-s, sn-post-code
 Once set to *False*, an equivalent `sample1` and `sample2` can be created with:
 ```
 ```python
 sample1 = sn.sql.select('sample_table', run=False)
 sample2 = sn.sql.select('sample_table')
 
 print(type(sample1))       #> <class 'str'>
 print(sample1 == sample2)  #> True
 ```

 ```{div} sn-pre-code-s, sn-post-code
 Because methods defer to {attr}`~snowmobile.core.sql.SQL.auto_run`
 in absence of an explicit argument, it can be executed off the same instance of
 <a class="fixture-sn" href="../index.html#fixture-sn"></a> with:
 ```
 ```python
 df_sample = sn.sql.select('sample_table', run=True)
 print(type(df_sample))  #> <class 'pandas.core.frame.DataFrame'>
 ```
 

````````

<hr class="sn-spacer-thick2">

(usage/sql/setting-nm-and-obj)=
### Setting {attr}`~snowmobile.SQL.nm` and {attr}`~snowmobile.SQL.obj`
<hr class="sn-green-thick-g">

````````{div} sn-tabbed-section

 ```````{tabbed} &nbsp;
 Most {class}`~snowmobile.SQL` methods need to know an in-warehouse object's name 
 ({attr}`~snowmobile.SQL.nm`) and type ({attr}`~snowmobile.SQL.obj`), which default 
 to *None* and *table* respectively. 
 
 ```{div} sn-pre-code-s
 These defaults are why we can write:
 ```
 ```python
 _ = sn.sql.drop('sample_table', run=False)
 ```
 
 ```{div} sn-pre-code-s
 Instead of:
 ```
 ```python
 _ = sn.sql.drop('sample_table', obj='table', run=False)
 ```
 
 <hr class="sn-spacer3">
 <hr class="sn-green-thin">
 
 In the same way as the `run` method argument and the [**auto_run**](using-auto-run) attribute, 
 {class}`~snowmobile.SQL` deffers to the values of its {attr}`~snowmobile.SQL.nm` and {attr}`~snowmobile.SQL.obj` 
 attributes in absence of valid strings passed through the `nm` and `obj` method arguments.
 
 ```{div} sn-pre-code-s
 Bringing these together and assuming a default instace of
 <a class="fixture-sn" href="../index.html#fixture-sn"></a>, the following can be run:
 ```
 ```python
 sn.sql.auto_run = False
 sn.sql.nm = 'sample_table'
 
 sample1 = sn.sql.select('sample_table')
 sample2 = sn.sql.select()
 df_sample = sn.sql.select(run=True)
 
 print(type(sample1))       #> <class 'str'>
 print(sample1 == sample2)  #> True
 print(type(df_sample))  #> <class 'pandas.core.frame.DataFrame'>
 ```
 
 ```````

````````

<hr class="sn-spacer-thick">

## Other
<hr class="sn-green-thick">

<br>

### Deprecations / Changes
<hr class="sn-grey">

{class}`~snowmobile.core.sql.SQL` will be merged with
{class}`~snowmobile.core.connection.Snowmobile` in a future release of 
{xref}`snowmobile` such that the methods of {class}`~snowmobile.core.sql.SQL` 
are accessible directly off
<a class="fixture-sn" href="../index.html#fixture-sn"></a> as opposed
to through its {attr}`~snowmobile.core.connection.Snowmobile.sql` attribute.

```{admonition} Note
 :class: error
 Re-factoring to account for this change will be an unrestricted 
 find/replace-all instances of `sn.sql` with `sn`
```

# NO RENDER BELOW THIS POINT
---

<br>

##### HEADER GOES HERE
<hr class="sn-green-thin-g">

``````{div} sn-tabbed-section

 `````{tabbed} &nbsp;
 CONTENT GOES HERE
 `````

 `````{tabbed} +
 MORE CONTENT GOES HERE
 `````     

```````

<hr class="sn-green-rounded-g">